# The Docker Image Construction Problem

Last edited: July 11, 2015

[Ryan J. O'Neil](mailto:roneil1@gmu.edu)  
Engineering @ [Yhat, Inc.](https://yhathq.com/)  
[SEOR](http://seor.gmu.edu) @ George Mason University

## Abstract

Nohthing too concrete.

## 1. Introduction

As cloud computing becomes more prevalent in the world of software and information technology, techniques for automated and repeatable cloning of computing environments grow increasingly important. One strategy for doing this is the creation of containers. A container is an isolated environment similar to a virtual machine, but with a much lighter weight architecture. Instead of setting another kernel on top of the host operating system, containers share the host's kernel. Container resources are isolated from the host, making it easy to run programs within their own custom environments, while still benefitting from low overhead.

### 1.1. A few notes about Linux containers

Running containers on a Unix host operating system is not a new thing. Development of the `chroot` system call for Unix and BSD dates back to the late 1970s and early '80s. This enabled creation of `chroot` "jails" in which system users could execute commands as the `root` user, but were only allowed to see a subset of the file system. Later, the FreeBSD project extended the concept into a command called `jail`. In 2005, Sun Microsystems coined the term container with the creation of "Solaris Containers," which were also built upon the `chroot` concept.

Recent developments in the Linux kernel have made container virtualization more convenient and powerful. Notable among these are the addition of "control groups" in the 2.6.24 kernel, which allow isolation of hardware resources to a group of processes, and "namespace isolation" in the 3.8 kernel, which isolates processes from seeing other processes on the same system. These advancements form the basis for modern container architectures, such as LXC, Docker, and Rocket, which allow containers to function as though they are their own operating systems and user spaces.

### 1.2. Motivation for the problem

The lure of low-overhead virtualization is proving extremely attractive for engineers across many disciplines in information technology. Containers provide easy replication of environments for software development, testing, and serving of infrastructure needs. They can be set up and torn down quickly without modifying their hosts, thus mapping naturally to distributed architectures. They allow one to capture complicated depedency structures for software in a single place, significantly speeding up development and testing times.

However, virtualization is no silver bullet. With the adoption of containers systems come new operational problems. This paper explores one of these which we call the Docker Image Construction Problem. While our focus will be on Docker's specific implementation of containers, the salient aspects of this problem can be mapped to other container implementations as well.

Consider a scenario many devops professionals are faced with regularly: the creation of environments in which to develop, test, and finally running prduction software. Depending on the scale of a given organization, there may be a great many of these. Each environment will require that a number of commands be run in order to create it. In Docker's terminology, an environment is an _image_, and an image is the result of running its associated commands. Each command requires a, potentially large, amount of time to run. The computing time required to create an image is the sum of the time required to run its commands.

For the purposes of this paper, we assume these commands are independent and that their order does not matter. There can, of course, be instances where that is not true. But those are refinements of the problem we present here. Examples of commands for creating these environment include:

* Download and install a language compiler and runtime environment.
* Set up a database and generate seed data for running tests.
* Upgrade an operating system and user space to the latest packages.
* Build a set of shared libraries for inclusion with a software package.
* Create binary packages for software distribution.
* Etc.

Any number of commands are possible to set up these environments, many of which require substantial computing time. In an organization that embraces the idea of continuous integration, these environments will be built up and torn down regularly, magnifying the cost of building them significantly.

Without virtualization or other techniques to mitigate the required computing time, the resources required will always be the sum of time required for each command in every image. That is, if we have a set $I$ of images to build and for each $i \in I$ there is a set of commands $c \in C_i$ where the time required by commmand $c$ is $t_c$, then our required compute time is:

$$\sum_{i \in I} \sum_{c \in C_i} t_c$$

It is common for commands to repeat across the creation of multiple images. An organization may standardize on a particular programming language or set of development tools. There may be a single data set created for both development and staging of a software product. Each image may use the same operating system as its base and expect that its kernel is regularly updated with the latest security patches.

If a container technology such as Docker is embraced, this sharing of commnands may allow one to reduce the compute time required to create a set of images. Docker, for instance, maintains an image cache. If a series of commands are run, it will reuse whatever it can safely determine has already been accomplished. Thus our goal in formulating and exploring this problem is build a set of required images in such as way that maximizes our use of the Docker cache. This will result in minimizing the requisite compute time.

### 1.3. Docker cache mechanics

An _image_ in Docker is constructed by running a series of _commands_. Commands are run in a specified order, and after each one Docker saves the changes made by that command to a _layer_. The final image is the result of applying each command's associated layer to some start image, such as a clean install of a Linux distribution.

For example, a given image may require the following commands be run in any order. We provide single-character aliases for them for the sake of convenience.

#### Image 1

|     | Command                                                  |
|-----|----------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.       |
| $B$ | Download a set of external dependencies.                 |
| $C$ | Set of a database schema and populate it with test data. |
 
Docker first runs command $A$ and saves the results of that command to its cache. It now run command $B$ and saves the results of $AB$ to the cache. Finally, it runs command $C$ and saves the reults of $ABC$ to the cache. Thus, layers are constructed and cached additively. Docker has $A$ and $AB$ in its cache, but not $B$ by itself.

After constructing Image 1, the Docker cache contains $\{A, AB, ABC\}$. So far we are building only a single image and each comand is unique, so this schedule obviously uses the minimal amount of compute time. Now say we want to build another version of the same image, but with a new kernel.

#### Image 2

|     | Command                                                  |
|-----|----------------------------------------------------------|
| $A$ | Install the Java compiler and runtime environment.       |
| $B$ | Download a set of external dependencies.                 |
| $C$ | Set of a database schema and populate it with test data. |
| $D$ | Upgrade the kernel to the latest version.                |

We request the image $ABCD$ from Docker. It first sees that $A$ already exists in its cache and need not be re-created. The same goes for $AB$ and $ABC$. $ABCD$ does not exist in the cache, so Docker runs command $D$ and applies it to $ABC$, saving the result in its cache.

In this case  we were able to build two unique images, while saving ourselves the time required to run commands $A$, $B$, and $C$ a second time. This is due to the mechanics of Docker's cache. Clearly, our current schedule for the creation of these two image uses the minimal amount of compute time.

However, say we are not so good at scheduling. Instead of putting the kernel upgrade last on our creation of Image 2, we decide to put it first. Now we request image $DABC$ from Docker. Docker looks at the first command and sees that it doe not have $D$ in its cache. It runs $D$ and caches its results. Similarly, it does not have $DA$, so it _(needlessly)_ runs $A$ again. The same goes for $B$ and $C$. At the end of the process, we have run $D$ once, and $A$, $B$, $C$ twice each. This results in using the maximum amount of compute time possible for creation of our two images.

It is easy to see that, due to the particulars of its cache mechanics, Docker is very sensitive to the order in which commands are run during image creation. Certain command schedules may waste much more compute time than others. Our goal is to always find the best schedule for creation of images.

## 2. Formulation

### 2.1. Individual image construction

### 2.2. Maximizing Docker cache usage

### 2.3. Model formulation

$$
\begin{array}{r l}
   \max        & z & = &\sum\limits_{i < j \in I}\sum\limits_{s = 1}^{\left\vert{C_i \cap C_j}\right\vert}\sum\limits_{c \in C_i \cap C_j} t_c y_{i,j,s,c} & & {\scriptsize\left(1\right)} \\
   \text{s.t.} &   &   & \sum\limits_{c \in C_i} x_{i,s,c} = 1 & \forall\, i \in I  & {\scriptsize\left(2\right)} \\
               &   &   & \sum\limits_{s = 1}^{\left\vert{C_i}\right\vert} x_{i,s,c} = 1 & \forall\, i \in I & {\scriptsize\left(3\right)} \\
               &   &   & y_{i,j,s,c} \leq x_{i,s,c} & \forall\, i < j \in I,\, s = 1,\dotsc,\left\vert{C_i \cap C_j}\right\vert,\, c \in C_i \cap C_j & {\scriptsize\left(4\right)} \\
               &   &   & y_{i,j,s,c} \leq x_{j,s,c} & \forall\, i < j \in I,\, s = 1,\dotsc,\left\vert{C_i \cap C_j}\right\vert,\, c \in C_i \cap C_j & {\scriptsize\left(5\right)} \\
               &   &   & \sum\limits_{c \in C_i \cap C_j} y_{i,j,s,c} \leq \sum\limits_{c \in C_i \cap C_j} y_{i,j,s-1,c} & \forall\, i < j \in I,\, s = 2,\dotsc,\left\vert{C_i \cap C_j}\right\vert & {\scriptsize\left(6\right)} \\
               &   &   & x_{i,s,c} \in \{0,1\} & & {\scriptsize\left(7\right)} \\
               &   &   & y_{i,j,s,c} \geq 0 & & {\scriptsize\left(8\right)} \\
\end{array}
$$

## 3. Numerical Example

## 4. Enhancements to the Formulation

### 4.1. Preprocessing

#### 4.1.1. Fully shared paths

#### 4.1.2. Unshared paths

### 4.2. Heuristics for generating initial feasible solutions

#### 4.2.1. Most common command

#### 4.2.2. Highest potential command

### 4.3. Decomposition

#### 4.3.1. Master problem

#### 4.3.2. Subproblem & Benders cuts

## 5. Next Steps